<a href="https://colab.research.google.com/github/rafaelrpq/classificadores/blob/main/classificadores_ajustado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset

from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    balanced_accuracy_score, confusion_matrix
)

from transformers import AutoModel
import warnings
import kagglehub
from tqdm import tqdm # For progress bars
from collections import defaultdict # To help with aggregation

# Import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning) # For sklearn metrics zero_division

# Configurar dispositivo (GPU se disponível)
device = torch.device ("cuda" if torch.cuda.is_available () else "cpu")
print (f"Usando o dispositivo: {device}")

# Hiperparametros
hyperparameters = {
    "SVC": [{"C": c} for c in [0.1, 1, 10, 100, 1000]],
    "MLPClassifier": [{"hidden_layer_sizes": hls, "max_iter" : 200, "random_state": 42} for hls in [(50,), (100,), (50, 50), (100, 100), (200, 200)]], # Added random_state
    "RandomForestClassifier": [{"n_estimators": n, "random_state" : 42} for n in [10, 50, 100, 200, 500]],
    "KNeighborsClassifier": [{"n_neighbors": k} for k in [1, 3, 5, 7, 9]]
}

# Transformação das imagens
transform = transforms.Compose ([
    transforms.Resize ((224, 224)),
    transforms.ToTensor (),
    transforms.Normalize (mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# --- Dataset Loading ---
# Carregar o dataset usando kagglehub
print("Downloading dataset...")
path = kagglehub.dataset_download("mudasirazhar/cyanotoxins-identification-dataset")
print("Path to dataset files:", path)

# !git clone https://github.com/iman2693/CTCB.git > /dev/null 2>&1
# path = "CTCB/dataset"

# Diretório das imagens
train_data_dir = os.path.join (path,"Dataset/Train")
test_data_dir = os.path.join (path,"Dataset/Test") # Assuming a 'Test' directory exists based on common dataset structures

# Carregar os conjuntos de dados de treino e teste (se houver diretórios separados)
# Se não houver Test directory, use train_test_split on the single directory
if not os.path.exists(test_data_dir):
     print("Test directory not found. Splitting train directory.")
     full_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
     test_ratio = 0.2
     # Use indices for splitting to avoid issues with ImageFolder and Subset later
     train_indices, test_indices = train_test_split(list(range(len(full_dataset))),
                                                    test_size=test_ratio,
                                                    random_state=42,
                                                    stratify=full_dataset.targets)
     train_dataset = Subset(full_dataset, train_indices)
     test_dataset = Subset(full_dataset, test_indices)
     # Need to get class names from the original full_dataset
     class_names = full_dataset.classes
     num_classes = len(class_names)

else:
    print("Found Train and Test directories.")
    train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
    test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)
    class_names = train_dataset.classes # Assume train_dataset has all classes
    num_classes = len(class_names)


print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print(f"Classes: {class_names}")


# Dataloaders para a extração inicial de features do conjunto COMPLETO de treino e teste
# Shuffle False for feature extraction to maintain order potentially, though not strictly necessary
# Use a larger batch size for extraction if memory allows
full_train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# KFold setup (apenas para o conjunto de TREINO inicial)
k_folds = 5
# Note: kf.split will return indices relative to the train_dataset subset if it's a Subset
# If it's the original ImageFolder, it will return indices relative to that.
# Our feature extraction step flattened this, so we need to be careful.
# Since we extract features *before* KFold, KFold should split indices of the *extracted features*.
# The number of samples for KFold should be len(X_train_full).
kf = KFold (n_splits=k_folds, shuffle=True, random_state=42)


# --- Feature Extraction Class ---
class FeatureExtractor :
    def __init__ (self, vit_model_name) :
        try:
             # Assuming the model is suitable for feature extraction with AutoModel
             self.model = AutoModel.from_pretrained(vit_model_name).to(device)
             print(f"Loaded {vit_model_name} as AutoModel.")
        except Exception as e:
             print(f"Error loading {vit_model_name} with AutoModel: {e}")
             print("Ensure the model checkpoint is compatible or try a different loading strategy.")
             raise e

        self.model.eval ()
        print(f"Feature extractor model loaded and set to evaluation mode.")

    def extract_features (self, dataloader):
        features = []
        labels = [] # Also return labels to keep them aligned with features
        print(f"Extracting features for {len(dataloader.dataset)} samples...")
        with torch.no_grad ():
            for inputs, targets in tqdm(dataloader, desc="Extracting Features"):
                inputs = inputs.to (device)
                outputs = self.model (inputs)

                # Attempt to get the CLS token or pooled output
                if hasattr(outputs, 'last_hidden_state') and outputs.last_hidden_state is not None:
                    # ViT-like models often use the first token (CLS)
                    cls_tokens = outputs.last_hidden_state[:, 0, :]
                elif hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
                     # Some models provide a pooled output
                     cls_tokens = outputs.pooler_output
                else:
                    # Fallback - might need adjustment based on specific model output
                    # print(f"Warning: Could not find standard feature outputs for {self.model.__class__.__name__}. Using first output tensor's first token.")
                    try:
                        # This is a risky fallback, verify for your model
                        cls_tokens = outputs[0][:, 0, :]
                    except Exception as e:
                         print(f"Error accessing fallback output: {e}")
                         print("Model output structure is unexpected. Check model documentation.")
                         raise e


                features.append (cls_tokens.cpu ().numpy ())  # Move to CPU and convert to numpy
                labels.append (targets.numpy ())

        return np.vstack (features), np.hstack (labels)


# --- Evaluation Function ---
def evaluate_metrics(y_true, y_pred, class_names=None, num_classes=None):
    """Calculates various classification metrics and CM components."""
    acc = accuracy_score(y_true, y_pred)

    # Handle cases with single class or limited classes in a subset/fold
    unique_classes_true = np.unique(y_true)
    unique_classes_pred = np.unique(y_pred)
    combined_unique_classes = np.unique(np.concatenate((unique_classes_true, unique_classes_pred)))

    precision, recall, f1 = np.nan, np.nan, np.nan
    if len(combined_unique_classes) >= 2:
        # Cannot calculate macro average if less than two unique classes are present in combined true/pred
        try:
            precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
            recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
            f1 = f1_score(y_true, y_pred, average="macro")
        except ValueError as e:
             # This might happen if one class has no true samples but predictions, etc.
             # print(f"Warning: Error calculating macro metrics: {e}. Setting to NaN.") # Too verbose
             precision, recall, f1 = np.nan, np.nan, np.nan


    balanced = balanced_accuracy_score(y_true, y_pred)

    # Confusion Matrix and per-class TP/TN/FP/FN
    # Use labels argument to ensure matrix shape corresponds to all possible classes
    all_possible_labels = np.arange(num_classes) if num_classes is not None else np.unique(np.concatenate((y_true, y_pred)))
    # Ensure labels array is sorted if you're passing it explicitly
    all_possible_labels = np.sort(all_possible_labels)
    cm = confusion_matrix(y_true, y_pred, labels=all_possible_labels)

    n_classes_cm = cm.shape[0]
    cm_metrics_per_class = {}

    # Ensure class names align with matrix indices
    current_class_names = class_names if class_names is not None and len(class_names) == n_classes_cm else [f'class_{i}' for i in range(n_classes_cm)]

    for i in range(n_classes_cm):
        class_label = current_class_names[i]
        # For class i (index i):
        TP_i = cm[i, i] # True class i, Predicted class i
        FN_i = np.sum(cm[i, :]) - TP_i # True class i, Predicted NOT class i (sum of row i excluding diagonal)
        FP_i = np.sum(cm[:, i]) - TP_i # True NOT class i, Predicted class i (sum of col i excluding diagonal)
        TN_i = np.sum(cm) - TP_i - FN_i - FP_i # True NOT class i, Predicted NOT class i (total - others)

        cm_metrics_per_class[class_label] = {'TP': int(TP_i), 'FN': int(FN_i), 'FP': int(FP_i), 'TN': int(TN_i)} # Ensure int

    return {
        "accuracy": float(acc), # Ensure float
        "precision_macro": float(precision) if precision is not np.nan else None,
        "recall_macro": float(recall) if recall is not np.nan else None,
        "f1_macro": float(f1) if f1 is not np.nan else None,
        "balanced_accuracy": float(balanced),
        "confusion_matrix": cm.tolist(), # Store as list
        "cm_metrics_per_class": cm_metrics_per_class
    }


# --- Main Execution ---

# Instanciando Extratores de Caracteristicas baseados em ViT Models
vit_extractors = {
    "DINO": FeatureExtractor("facebook/dino-vitb8"),
    # "ViT-Base": FeatureExtractor("google/vit-base-patch16-224"), # Uncomment to include
    # "ViT-Large": FeatureExtractor("google/vit-large-patch16-224"), # Uncomment to include
}

# Instanciando Classificadores (using sklearn class objects directly)
classifiers_map = {
    'SVM': SVC,
    'MLP': MLPClassifier,
    'RandomForest': RandomForestClassifier,
    'KNN': KNeighborsClassifier,
}

# --- 1. Extract features from the FULL train and test datasets ONCE ---
print("\n" + "="*60)
print("--- Initial Feature Extraction (Full Train and Test) ---")
print("="*60)
feature_data = {} # Store features and labels per extractor

# Use train_dataset and test_dataset directly for feature extraction
for extractor_name, extractor in vit_extractors.items():
    print(f"\nUsing extractor: {extractor_name}")
    full_train_loader_for_extraction = DataLoader(train_dataset, batch_size=64, shuffle=False)
    test_loader_for_extraction = DataLoader(test_dataset, batch_size=64, shuffle=False)

    X_train_full, y_train_full = extractor.extract_features(full_train_loader_for_extraction)
    X_test_full, y_test_full = extractor.extract_features(test_loader_for_extraction)

    feature_data[extractor_name] = {
        'X_train_full': X_train_full,
        'y_train_full': y_train_full,
        'X_test_full': X_test_full,
        'y_test_full': y_test_full,
    }
    print(f"Extracted features shape (Train): {X_train_full.shape}")
    print(f"Extracted features shape (Test): {X_test_full.shape}")

# --- 2. K-Fold Cross-Validation on Training Data ---
print("\n" + "="*60)
print("--- Starting K-Fold Cross-Validation on Training Data ---")
print("="*60)
kfold_results = [] # Store results for each fold, classifier, and params

# K-Fold split on the indices of the extracted full training features
for extractor_name, data in feature_data.items():
    X_train_full = data['X_train_full']
    y_train_full = data['y_train_full']

    print(f"\n--- K-Fold for Extractor: {extractor_name} ---")

    # kf.split splits indices 0..N-1 where N is the first argument's length
    for fold_idx, (train_indices_fold, val_indices_fold) in enumerate(kf.split(X_train_full)):
        print(f"\nProcessing Fold {fold_idx + 1}/{k_folds}")

        # Get the features and labels for this fold using the indices
        X_train_fold = X_train_full[train_indices_fold]
        y_train_fold = y_train_full[train_indices_fold]
        X_val_fold = X_train_full[val_indices_fold]
        y_val_fold = y_train_full[val_indices_fold]

        # Iterate through each classifier
        for clf_name, clf_class in classifiers_map.items():
            clf_hyperparameters = hyperparameters.get(clf_class.__name__, [])

            if not clf_hyperparameters:
                 continue

            for params in tqdm(clf_hyperparameters, desc=f"  Fold {fold_idx+1} {clf_name} Params"):
                try:
                    clf = clf_class(**params)
                    clf.fit(X_train_fold, y_train_fold)
                    y_pred_fold = clf.predict(X_val_fold)
                    metrics = evaluate_metrics(y_val_fold, y_pred_fold, class_names=class_names, num_classes=num_classes)

                    kfold_results.append({
                        'extractor': extractor_name,
                        'fold_idx': fold_idx + 1,
                        'classifier': clf_name,
                        'params': params,
                        'metrics': metrics
                        # Note: y_true/y_pred not strictly needed for selection, saving memory
                    })
                except Exception as e:
                     print(f"\nError training/predicting {clf_name} with params {params} in fold {fold_idx+1}: {e}")
                     kfold_results.append({
                         'extractor': extractor_name,
                         'fold_idx': fold_idx + 1,
                         'classifier': clf_name,
                         'params': params,
                         'error': str(e)
                     })

print("\n--- K-Fold Cross-Validation Finished ---")
print(f"Total K-Fold results stored (including errors): {len(kfold_results)}")


# --- 3. Analyze K-Fold Results and Select Best Hyperparameters ---
print("\n" + "="*60)
print("--- Analyzing K-Fold Results and Selecting Best Models ---")
print("="*60)

aggregated_kfold_metrics = defaultdict(lambda: defaultdict(list))

for result in kfold_results:
    if 'error' in result:
        continue

    extractor = result['extractor']
    classifier = result['classifier']
    # Convert params dict to a hashable format (string representation)
    # Use json.dumps for a more stable representation than str(), handling dict order
    try:
         import json
         params_key = json.dumps(result['params'], sort_keys=True)
    except:
         params_key = str(result['params']) # Fallback if json fails

    for metric_name, metric_value in result['metrics'].items():
        if isinstance(metric_value, (int, float)) or (metric_value is None):
             if metric_value is not None: # Only average non-None values
                aggregated_kfold_metrics[(extractor, classifier, params_key)][metric_name].append(metric_value)


average_kfold_metrics = {}
best_params_per_clf_extractor = {}

selection_metric = 'balanced_accuracy' # Metric used to select the best hyperparameters

print(f"Selecting best hyperparameters based on average '{selection_metric}' across folds.")

# Need to map params_key back to original dict structure
# Create a mapping from the string representation to the actual dictionary
params_key_to_dict = {}
for res in kfold_results:
    if 'error' not in res:
        try:
            import json
            params_key_json = json.dumps(res['params'], sort_keys=True)
            params_key_to_dict[params_key_json] = res['params']
        except:
            params_key_str = str(res['params']) # Fallback if json fails
            params_key_to_dict[params_key_str] = res['params']


for (extractor, classifier, params_key), metrics_list_dict in aggregated_kfold_metrics.items():
    average_metrics = {
        metric_name: np.nanmean(metric_values)
        for metric_name, metric_values in metrics_list_dict.items()
    }

    # average_kfold_metrics is now correctly keyed by (extractor, classifier, params_key)
    average_kfold_metrics[(extractor, classifier, params_key)] = average_metrics

    current_best_info = best_params_per_clf_extractor.get((extractor, classifier))
    current_combination_score = average_metrics.get(selection_metric, np.nan) # Get score, default to NaN if metric missing

    # Update best if current score is better (handling NaN: NaN is never > a number, nor is a number > NaN. Treat NaN as worst.)
    if current_best_info is None or np.isnan(current_best_info['avg_score']) or (not np.isnan(current_combination_score) and current_combination_score > current_best_info['avg_score']):
        # Retrieve the original params dict using the params_key
        original_params = params_key_to_dict.get(params_key)
        best_params_per_clf_extractor[(extractor, classifier)] = {
            'params': original_params, # Store the actual params dict here
            'avg_score': current_combination_score
        }


print("\n--- K-Fold Average Metrics Summary (Best per Classifier) ---")
# Sort by score, treat NaN as a low value (-1 or negative infinity) for sorting
sorted_best_kfold = sorted(best_params_per_clf_extractor.items(),
                           key=lambda item: item[1]['avg_score'] if not np.isnan(item[1]['avg_score']) else -float('inf'),
                           reverse=True)

for (extractor, classifier), best_info in sorted_best_kfold:
     best_params = best_info['params']
     avg_score = best_info['avg_score']

     # === FIX ===
     # Use the params_key (string representation) to lookup in average_kfold_metrics
     try:
         import json
         best_params_key = json.dumps(best_params, sort_keys=True)
     except:
         best_params_key = str(best_params)

     full_avg_metrics = average_kfold_metrics.get((extractor, classifier, best_params_key), {})
     # === END FIX ===

     print(f"\nBest K-Fold Result for Extractor: {extractor}, Classifier: {classifier}")
     print(f"  Selected Params: {best_params}")
     print(f"  Avg {selection_metric}: {avg_score:.4f}" if not np.isnan(avg_score) else f"  Avg {selection_metric}: N/A (all folds failed or metric NaN)")
     # Safely print average metrics, handling potential missing keys gracefully
     print(f"  Avg Metrics: Accuracy={full_avg_metrics.get('accuracy', np.nan):.4f}, Precision={full_avg_metrics.get('precision_macro', np.nan):.4f}, Recall={full_avg_metrics.get('recall_macro', np.nan):.4f}, F1={full_avg_metrics.get('f1_macro', np.nan):.4f}")


# --- 4. Final Evaluation on the Test Set (Using Selected Best Hyperparameters) ---
print("\n" + "="*60)
print("--- Starting Final Evaluation on Test Set (Using Selected Best Models) ---")
print("="*60)
final_test_results = []

# For each extractor (only DINO in this run)
for extractor_name, data in feature_data.items():
    X_train_full = data['X_train_full']
    y_train_full = data['y_train_full']
    X_test_full = data['X_test_full']
    y_test_full = data['y_test_full']

    print(f"\n--- Testing with Extractor: {extractor_name} ---")

    # For each classifier, get the best params found
    for clf_name, clf_class in classifiers_map.items():
         best_info = best_params_per_clf_extractor.get((extractor_name, clf_name))

         # Also check if the best score found was NaN, if so, maybe skip testing this config
         if best_info is None or best_info['params'] is None or np.isnan(best_info.get('avg_score', np.nan)):
              print(f"  No valid best parameters found for {clf_name} with {extractor_name} (or avg score was NaN). Skipping final test.")
              continue

         params = best_info['params']
         print(f"  Testing {clf_name} with Selected Best Params: {params}")

         try:
            # Instantiate and train the classifier on the *entire* initial training set
            clf = clf_class(**params)
            clf.fit(X_train_full, y_train_full)

            # Predict on the held-out test set
            y_pred_test = clf.predict(X_test_full)

            # Evaluate metrics for the final test set
            test_metrics = evaluate_metrics(y_test_full, y_pred_test, class_names=class_names, num_classes=num_classes)

            # Store final test results (only one entry per extractor/classifier now)
            final_test_results.append({
                'extractor': extractor_name,
                'classifier': clf_name,
                'params': params, # Store the best params used
                'metrics': test_metrics,
                'y_true': y_test_full.tolist(), # Keep for CM plotting
                'y_pred': y_pred_test.tolist() # Keep for CM plotting
            })
         except Exception as e:
             print(f"\nError training/predicting selected {clf_name} with params {params} on test set: {e}")
             final_test_results.append({
                 'extractor': extractor_name,
                 'classifier': clf_name,
                 'params': params,
                 'error': str(e)
             })


print("\n--- Final Evaluation on Test Set Finished ---")
print(f"Total Final Test results stored (selected models): {len(final_test_results)}")


# --- 5. Display Final Test Results (with Plotting) ---
print("\n" + "="*60)
print("--- Summarizing Final Test Results (Selected Models) ---")
print("="*60)

if not final_test_results:
    print("No final test results to display.")
else:
    for result in final_test_results:
        print("\n" + "=" * 60)
        print(f"Extractor: {result.get('extractor', 'N/A')}")
        print(f"Classifier: {result.get('classifier', 'N/A')}")
        print(f"Parameters (Selected by K-Fold): {result.get('params', 'N/A')}")
        print("-" * 60)

        if 'error' in result:
             print(f"Error during final test: {result['error']}")
             continue

        metrics = result['metrics']

        # --- Print Scalar Metrics ---
        print(f"  Metrics on Test Set:")
        # Use .get() and handle None/NaN gracefully
        acc = metrics.get('accuracy')
        bal_acc = metrics.get('balanced_accuracy')
        prec = metrics.get('precision_macro')
        rec = metrics.get('recall_macro')
        f1_m = metrics.get('f1_macro')

        print(f"    Accuracy: {acc:.4f}" if acc is not None else "    Accuracy: N/A")
        print(f"    Balanced Accuracy: {bal_acc:.4f}" if bal_acc is not None else "    Balanced Accuracy: N/A")
        print(f"    Precision (macro): {prec:.4f}" if prec is not None else "    Precision (macro): N/A")
        print(f"    Recall (macro): {rec:.4f}" if rec is not None else "    Recall (macro): N/A")
        print(f"    F1-Score (macro): {f1_m:.4f}" if f1_m is not None else "    F1-Score (macro): N/A")


        # --- Plot Confusion Matrix ---
        cm_list = metrics.get('confusion_matrix')
        if cm_list is not None and class_names is not None: # Ensure class_names are available for plotting
            cm_np = np.array(cm_list)

            plt.figure(figsize=(8, 6))
            sns.heatmap(cm_np,
                        annot=True, fmt='d', cmap='Blues',
                        xticklabels=class_names, yticklabels=class_names)
            plt.xlabel('Predicted Label')
            plt.ylabel('True Label')
            # Include params in the title, maybe as a string summary
            # Use the actual params dict which is stored in the result
            params_dict_for_title = result.get('params', {})
            params_str_title = ", ".join([f"{k}={v}" for k, v in params_dict_for_title.items()])

            plt.title(f'Confusion Matrix: {result["classifier"]} ({result["extractor"]})\nParams: {params_str_title}')
            plt.tight_layout() # Adjust layout to prevent labels overlapping
            plt.show() # Display the plot
        elif cm_list is not None:
             print("\n  Confusion Matrix (cannot plot without class names):")
             print(np.array(cm_list))


        # --- Print Per-Class TP/TN/FP/FN ---
        cm_metrics_per_class = metrics.get('cm_metrics_per_class')
        if cm_metrics_per_class:
            print("\n  Per-Class TP/TN/FP/FN on Test Set:")
            sorted_class_labels = sorted(cm_metrics_per_class.keys()) # Sort by class name/key
            for class_label in sorted_class_labels:
                cm_vals = cm_metrics_per_class[class_label]
                tp = cm_vals.get('TP', 'N/A')
                tn = cm_vals.get('TN', 'N/A')
                fp = cm_vals.get('FP', 'N/A')
                fn = cm_vals.get('FN', 'N/A')
                print(f"    {class_label}: TP={tp}, TN={tn}, FP={fp}, FN={fn}")

        print("=" * 60) # Separator for next result

print("\n--- All Final Test Results Displayed ---")

# --- 6. Identify and Display Overall Best Result ---
print("\n" + "="*60)
print("--- Overall Best Result on Test Set ---")
print("="*60)

best_result_entry = None
highest_balanced_accuracy = -1.0 # Initialize with a value lower than any possible score

# Iterate through the final test results to find the best one
for result in final_test_results:
    # Skip results with errors or missing metrics
    if 'error' in result or result.get('metrics') is None:
        continue

    current_balanced_accuracy = result['metrics'].get('balanced_accuracy')

    # Compare, handling None values (None is treated as worse than any number)
    if current_balanced_accuracy is not None:
        if best_result_entry is None or current_balanced_accuracy > highest_balanced_accuracy:
            highest_balanced_accuracy = current_balanced_accuracy
            best_result_entry = result

# Print the details of the best result found
if best_result_entry:
    print(f"\nOverall Best Model configuration based on Test Set Balanced Accuracy ({highest_balanced_accuracy:.4f}):")
    print("-" * 60)
    print(f"Extractor: {best_result_entry['extractor']}")
    print(f"Classifier: {best_result_entry['classifier']}")
    print(f"Parameters (Selected by K-Fold): {best_result_entry['params']}")
    print("-" * 60)
    print("Metrics on Test Set for the Best Model:")

    metrics = best_result_entry['metrics']
    acc = metrics.get('accuracy')
    bal_acc = metrics.get('balanced_accuracy')
    prec = metrics.get('precision_macro')
    rec = metrics.get('recall_macro')
    f1_m = metrics.get('f1_macro')

    print(f"  Accuracy: {acc:.4f}" if acc is not None else "  Accuracy: N/A")
    print(f"  Balanced Accuracy: {bal_acc:.4f}" if bal_acc is not None else "  Balanced Accuracy: N/A")
    print(f"  Precision (macro): {prec:.4f}" if prec is not None else "  Precision (macro): N/A")
    print(f"  Recall (macro): {rec:.4f}" if rec is not None else "  Recall (macro): N/A")
    print(f"  F1-Score (macro): {f1_m:.4f}" if f1_m is not None else "  F1-Score (macro): N/A")

    # You could optionally re-plot the CM or print CM details here if desired,
    # but it's often sufficient to just show the scalar metrics for the "best".

else:
    print("Could not determine the overall best result (perhaps all runs failed or had no valid metrics).")

print("="*60)

Usando o dispositivo: cuda
Path to dataset files: /home/rafaelquevedo/.cache/kagglehub/datasets/mudasirazhar/cyanotoxins-identification-dataset/versions/1
Found Train and Test directories.
Train dataset size: 2377
Test dataset size: 595
Classes: ['Anabaena', 'Aphanizomenon', 'Chroococcales', 'Cylindrospermopsis', 'Dolichospermum', 'Microcystis', 'Nostoc', 'Oscillatoria', 'Phormidium', 'Planktothrix', 'Raphidiopsis', 'Rivularia', 'Synechococcus']


Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vitb8 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded facebook/dino-vitb8 as AutoModel.
Feature extractor model loaded and set to evaluation mode.

--- Initial Feature Extraction (Full Train and Test) ---

Using extractor: DINO
Extracting features for 2377 samples...


Extracting Features:   0%|          | 0/38 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB. GPU 0 has a total capacity of 1.94 GiB of which 74.06 MiB is free. Process 17435 has 27.88 MiB memory in use. Process 2987 has 10.04 MiB memory in use. Including non-PyTorch memory, this process has 1.32 GiB memory in use. Of the allocated memory 1.23 GiB is allocated by PyTorch, and 45.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)